# Script Machine Debugging Tool (Libbitcoin c++)

<br>
<img src="images/script_evaluation_bip_16_141.jpg" alt="drawing" style="" width="800px"/>


<hr style="border: 0.5px dashed #000;">

In [2]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 

// Libbitcoin-Script machine helper.
#include "supporting_scripts/script_machine.cpp"

In file included from input_line_14:1:
./supporting_scripts/script_machine.cpp:3:10: error: redefinition of 'run_script_on_program'
bc::code run_script_on_program(bc::machine::program& current_program,
         ^
./supporting_scripts/script_machine.cpp:3:10: note: previous definition is here
bc::code run_script_on_program(bc::machine::program& current_program,
         ^
In file included from input_line_14:1:
./supporting_scripts/script_machine.cpp:91:10: error: redefinition of 'evaluate_in_out_p2sh_witness_runs'
bc::code evaluate_in_out_p2sh_witness_runs(const bc::chain::script
         ^
./supporting_scripts/script_machine.cpp:91:10: note: previous definition is here
bc::code evaluate_in_out_p2sh_witness_runs(const bc::chain::script
         ^


Interpreter Error: 

### 1. Previous transaction output (UTXO).

**Previous output script** (fetch by `txid` & `index`)

In [3]:
!bx fetch-tx 86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363 -f json | jq -r  ".transaction.outputs[8].script" | bx script-encode


76a914a0aef4980069732dba8b51f668a504a95f37bda588ac


In [4]:
auto previous_output_script_data = bc::to_chunk(
  bc::base16_literal("76a914a0aef4980069732dba8b51f668a504a95f37bda588ac"));
bc::chain::script previous_output_script;
previous_output_script.from_data(previous_output_script_data, false);


**Previous output amount** (fetch by `txid` & `index`)

In [5]:
!bx fetch-tx 86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363 -f json | jq -r  ".transaction.outputs[8].value"


499543


In [6]:
uint64_t previous_output_amount = 499543;


### 2. Spending transaction.

In [7]:
// Paste your raw transaction data.
auto spending_transaction_data = bc::to_chunk(
  bc::base16_literal("010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86080000006b48304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e012103aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67ffffffff01079d070000000000160014757d9e1576e8b890abdeba9971f7238ab9aa0a8a00000000"));
bc::chain::transaction spending_transaction;
std::cout << spending_transaction.from_data(spending_transaction_data, true, true); // wire=true, witness=true


1

@0x105d85010

**Index of spending input.**

In [8]:
uint32_t input_index = 0;

### 3. Evaluate input, output, p2sh and witness script runs.

Run following cell to evaluate the `previous output` | `input` script(s).

In [9]:
bc::code ec = evaluate_in_out_p2sh_witness_runs(previous_output_script, previous_output_amount,
  spending_transaction, input_index, bc::machine::rule_fork::all_rules);
std::cout << ec.message() << std::endl;


=========== Input script evaluation ===========

> Operation: 0
[304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e01]

> Stack after operation: 0
[304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e01]

>> Operation: 1
[03aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67]

>> Stack after operation: 1
[03aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67]
[304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e01]

=========== Output script evaluation ==========

> Operation: 0
dup

> Stack after operation: 0
[03aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67]
[03aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67]
[304502210093fee9cea2af3245f50f656fb29cef95faed4